<h1> Import libraries here </h1>

In [88]:
import sklearn as sk
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error
from sklearn import linear_model

## For Data visualisation 
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
import seaborn as sn
# from adspy_shared_utilities import plot_class_regions_for_classifier_subplot #Save to git first


<h2>1. DATA PROCESSING</h2>

<h3>Functions</h3>

In [89]:
def split(x, y, train = 0.8, test = 0): 
    valid = 1 - train - test
    x_trainy, x_valid, y_trainy, y_valid = train_test_split(x, y, train_size = (train+test), test_size = valid)
    if test > 0:
        x_train, x_test, y_train, y_test = (
            train_test_split(x_trainy, y_trainy, train_size = train/(train+test), test_size = test/(train+test)))
    else:
        x_train, y_train =  x_trainy, y_trainy
        x_train, y_train =  x_trainy, y_trainy
        x_test, y_test = 0, 0
    return x_train, x_valid, x_test, y_train, y_valid, y_test

def score_RandomForest(x_train, x_valid, y_train, y_valid):
    model = RandomForestRegressor(n_estimators=100)
    model.fit(x_train, y_train)
    preds = model.predict(x_valid)
    return model, mean_absolute_error(y_valid, preds)

<h3> Preprocessing </h3>

In [112]:
dataPure = pd.read_excel("Digital_Data.xlsx")
data = dataPure.copy()
data = data.replace(to_replace = 0, value = np.NaN)
label_encoder = LabelEncoder()
s = (data.dtypes == 'object')
cat_cols = list(s[s].index)
for col in cat_cols:
    data[col] = label_encoder.fit_transform(data[col].astype(str))

##Dropping Features HERE
data = data.drop(["Project Ref No.", "Blade length (m)", "Tower height (m)","Built duration"], axis = 1)

data_sets = [data.drop(["Single Blade Weight (te)"], axis = 1), data.drop(["Nacelle Weights"], axis = 1)]
X = []
Y = []
for i in range(2):
    data_sets[i] = data_sets[i].dropna()
    x = data_sets[i].drop(columns = data_sets[i].columns[4])
    y = data_sets[i][data_sets[i].columns[4]]
    x_train, x_valid, x_test, y_train, y_valid, y_test = split(x,y)
    X.append([x_train, x_valid, x_test])
    Y.append([y_train, y_valid, y_test])
print(Y[1][1])

10     9.60
17    20.00
14    20.00
26    16.59
13    16.00
39    21.00
1     22.00
37    21.00
54     5.00
25    20.00
12    14.00
35     6.30
Name: Single Blade Weight (te), dtype: float64


<h3>Random Forrest</h3>

In [128]:
for i in range(2):
    model, mae = score_RandomForest(X[i][0], X[i][1], Y[i][0], Y[i][1])
    print(mae)

16.599079999999997
3.0551250000000008
